In [1]:
from simulate_game import simulate_game
from competitive_sudoku.sudoku import load_sudoku

import importlib
import sys
import io
import os
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from datetime import datetime

from tqdm import tqdm

In [2]:
# Winner 0 if it was a draw, 1 if player 1 won, 2 if player 2 won
# Reason:
# "regular" if it was played till the end
# "taboo" if a taboo move was played
# "invalid" if invalid move was played
# "illegal" if illegal move was played
# "no move" if no move was supplied


def run_simulation(first_player: str, second_player: str, board_name: str, time: float) -> set:
    player1 = importlib.import_module(first_player + '.sudokuai').SudokuAI()
    player2 = importlib.import_module(second_player + '.sudokuai').SudokuAI()
    player1.player_number = 1
    player2.player_number = 2
    
    # We do not check for specific ai, just give it to all of them
    player1.solve_sudoku_path = 'bin\\solve_sudoku.exe'
    player2.solve_sudoku_path = 'bin\\solve_sudoku.exe'
    
    board = load_sudoku(board_name)
    
    # Note for future, we do not clean up files
    
    old_stdout = sys.stdout
    sys.stdout = buffer = io.StringIO()
    
    simulate_game(board, player1, player2, solve_sudoku_path='bin\\solve_sudoku.exe', calculation_time=time)
    
    sys.stdout = old_stdout
    result = buffer.getvalue()
    
    last_line = result.splitlines()[-1]
    splitted_line = last_line.split(".")
    
    if len(splitted_line) == 2:
        winner = {"The game ends in a draw": 0, "Player 1 wins the game":1, 
                  "Player 2 wins the game":2}[splitted_line[0]]
        reason = "regular"
    
    else:
        winner = {" The game ends in a draw": 0, " Player 1 wins the game":1, 
                  " Player 2 wins the game":2}[splitted_line[1]]
        reason = {"taboo move": "taboo", "valid move": "invalid", "legal move": "illegal", 
                  "s supplied": "no move"}[splitted_line[0][-10:]]
    
    return winner, reason

In [3]:
from distutils.dir_util import copy_tree

def create_new_file(C, t, s):

    copy_tree("MC_tests//team41_A3_MC_2_False_max", f"MC_tests//team41_A3_MC_{C}_{t}_{s}")
    os.remove(f"MC_tests//team41_A3_MC_{C}_{t}_{s}//sudokuai.py")

    with open("MC_tests//team41_A3_MC_2_False_max//sudokuai.py") as old_file:
        with open(f"MC_tests//team41_A3_MC_{C}_{t}_{s}//sudokuai.py", "w") as new_file:
            for line in old_file:
                if line == "global_C = 2\n":
                    new_file.write(f"global_C = {C}\n")

                elif line == "global_total = False\n":
                    new_file.write(f"global_total = {t}\n")

                elif line == "global_selection = \"max\"\n":
                    new_file.write(f"global_selection = \"{s}\"\n")

                else:
                    new_file.write(line)

In [4]:
Cs = [0, 2, 5, 10]
totals = [False, True]
selections = ["max", "robust"]

In [ ]:
boards = os.listdir("boards")
df = pd.DataFrame(columns = ["board", "starting", "C", "totals", "selection", "result", "reason"])

for setup in tqdm(list(itertools.product(Cs, totals, selections))):
    if setup != (2,False,"max"):
        create_new_file(setup[0], setup[1], setup[2])
        
for board in tqdm(boards):
    for setup in tqdm(list(itertools.product(Cs, totals, selections))):
        current_time = datetime.now().strftime("%H:%M:%S")
        print("Current Time =", current_time)

        winner, reason = run_simulation(f"MC_tests.team41_A3_MC_{setup[0]}_{setup[1]}_{setup[2]}","random_player", f'boards//{board}', 0.5)
        result = {0: 0, 1:1, 2:-1}[winner]
        new_row = pd.DataFrame({"board": board[:-4], "starting": True, "C": setup[0], "totals": setup[1],
                                "selection": setup[2], "result": result, "reason": reason}, index=[0])

        df = pd.concat([df, new_row]).reset_index(drop = True)


        current_time = datetime.now().strftime("%H:%M:%S")
        print("Current Time =", current_time)

        winner, reason = run_simulation("random_player", f"MC_tests.team41_A3_MC_{setup[0]}_{setup[1]}_{setup[2]}", f'boards//{board}', 0.5)
        result = {0: 0, 1:-1, 2:1}[winner]
        result = {0: 0, 1:1, 2:-1}[winner]
        new_row = pd.DataFrame({"board": board[:-4], "starting": True, "C": setup[0], "totals": setup[1],
                                "selection": setup[2], "result": result, "reason": reason}, index=[0])

        df = pd.concat([df, new_row]).reset_index(drop = True)

        df.to_csv(f'MC_tests//results4.csv', index = False)

In [ ]:
df = pd.DataFrame(columns = ["board", "starting", "C", "totals", "selection", "result", "reason"])
boards = os.listdir("boards")

for setup in tqdm(list(itertools.product(Cs, totals, selections))):
    if setup != (2,False,"max"):
        create_new_file(setup[0], setup[1], setup[2])
    
for i in tqdm(range(len(boards))):
    if i>6:
        board = boards[i]
        for setup in tqdm(list(itertools.product(Cs, totals, selections))):
            current_time = datetime.now().strftime("%H:%M:%S")
            print("Current Time =", current_time)

            winner, reason = run_simulation(f"MC_tests.team41_A3_MC_{setup[0]}_{setup[1]}_{setup[2]}","random_player", f'boards//{board}', 0.5)
            result = {0: 0, 1:1, 2:-1}[winner]
            new_row = pd.DataFrame({"board": board[:-4], "starting": True, "C": setup[0], "totals": setup[1],
                                    "selection": setup[2], "result": result, "reason": reason}, index=[0])

            df = pd.concat([df, new_row]).reset_index(drop = True)


            current_time = datetime.now().strftime("%H:%M:%S")
            print("Current Time =", current_time)

            winner, reason = run_simulation("random_player", f"MC_tests.team41_A3_MC_{setup[0]}_{setup[1]}_{setup[2]}", f'boards//{board}', 0.5)
            result = {0: 0, 1:-1, 2:1}[winner]
            result = {0: 0, 1:1, 2:-1}[winner]
            new_row = pd.DataFrame({"board": board[:-4], "starting": True, "C": setup[0], "totals": setup[1],
                                    "selection": setup[2], "result": result, "reason": reason}, index=[0])

            df = pd.concat([df, new_row]).reset_index(drop = True)

            df.to_csv(f'MC_tests//results5.csv', index = False)

  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

Current Time = 15:49:56
Current Time = 15:50:45



  6%|█████▏                                                                             | 1/16 [01:34<23:44, 94.96s/it]

Current Time = 15:51:31
Current Time = 15:52:21



 12%|██████████▍                                                                        | 2/16 [03:12<22:34, 96.74s/it]

Current Time = 15:53:09
Current Time = 15:53:55



 19%|███████████████▌                                                                   | 3/16 [04:47<20:44, 95.73s/it]

Current Time = 15:54:43


In [ ]:
df = pd.read_csv("MC_tests\\results3.csv")
df2 = pd.read_csv("MC_tests\\results5.csv")

In [ ]:
wi = round(len(df[df["result"]==1])/len(df),2)
print(f"Overall Winrate: {wi}")

cx = []
cy = []
for c in [0, 2, 5, 10]:
    cx.append(str(c))
    cy.append(round(len(df[(df["C"]==c) & (df["result"]==1)])/len(df[df["C"]==c]),2))
    
tox = []
toy = []
for to in [True, False]:
    tox.append({True: "Totals", False: "Wins"}[to])
    toy.append(round(len(df[(df["totals"]==to) & (df["result"]==1)])/len(df[df["totals"]==to]),2))
    
sex = []
sey = []
for se in ["max", "robust"]:
    sex.append(se)
    sey.append(round(len(df[(df["selection"]==se) & (df["result"]==1)])/len(df[df["selection"]==se]),2))
    
six = []
siy = []
for si in ["2x2", "2x3", "3x3", "3x4", "4x4"]:
    six.append(si)
    siy.append(round(len(df[(df["board"].str.contains(si)) & (df["result"]==1)])/len(df[df["board"].str.contains(si)]),2))

In [ ]:
fig, axs = plt.subplots(1, 4, sharey = True, figsize=(20,5))
fig.suptitle("Winrates Using Different Settings", fontsize=25)

axs[0].bar(cx, cy, color = "#6BC319")
axs[0].set_xlabel("C", fontsize=18)
axs[0].set_ylabel("Winrate", fontsize=18)
axs[0].tick_params(axis="both", labelsize=14)

axs[0].set_ylim(top=0.6)

axs[1].bar(tox, toy, color = "#6BC319")
axs[1].set_xlabel("Total Values", fontsize=18)
axs[1].tick_params(axis="both", labelsize=14)

axs[2].bar(sex, sey, color = "#6BC319")
axs[2].set_xlabel("Selection Method", fontsize=18)
axs[2].tick_params(axis="both", labelsize=14)

axs[3].bar(six, siy, color = "#6BC319")
axs[3].set_xlabel("Boardsize", fontsize=18)
axs[3].tick_params(axis="both", labelsize=14)

for i in range(4):
    for p in axs[i].patches:
        axs[i].annotate(str(p.get_height()), (p.get_x() + 0.25, p.get_height() + 0.01), fontsize=14)

box_style=dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(-18.55, 0.55, "n = 96 per possibility",{'color':'black','weight':'heavy','size':12},bbox=box_style)
plt.text(-12.3, 0.55, "n = 192 per possibility",{'color':'black','weight':'heavy','size':12},bbox=box_style)   
plt.text(-6.5, 0.55, "n = 192 per selection method",{'color':'black','weight':'heavy','size':12},bbox=box_style)   
plt.text(-0.2, 0.56, "n = 64 per board n = 128 for 3x3",{'color':'black','weight':'heavy','size':11},bbox=box_style)
 

plt.savefig(f"MC_tests\\MC_test_result.png")